In [30]:
import pandas as pd
import glob
import pycountry
import pickle
import re
from tqdm import tqdm
import gensim
import nltk
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim import corpora
import spacy
import matplotlib.pyplot as plt

In [31]:
# list of all bibtext files
file_list = glob.glob('bibtex_files/**/*.bib')
file_list

['bibtex_files/WRR/savedrecs.bib',
 'bibtex_files/WRR/savedrecs-29.bib',
 'bibtex_files/WRR/savedrecs-15.bib',
 'bibtex_files/WRR/savedrecs-14.bib',
 'bibtex_files/WRR/savedrecs-28.bib',
 'bibtex_files/WRR/savedrecs-16.bib',
 'bibtex_files/WRR/savedrecs-9.bib',
 'bibtex_files/WRR/savedrecs-8.bib',
 'bibtex_files/WRR/savedrecs-17.bib',
 'bibtex_files/WRR/savedrecs-13.bib',
 'bibtex_files/WRR/savedrecs-12.bib',
 'bibtex_files/WRR/savedrecs-10.bib',
 'bibtex_files/WRR/savedrecs-38.bib',
 'bibtex_files/WRR/savedrecs-39.bib',
 'bibtex_files/WRR/savedrecs-11.bib',
 'bibtex_files/WRR/savedrecs-34.bib',
 'bibtex_files/WRR/savedrecs-20.bib',
 'bibtex_files/WRR/savedrecs-3.bib',
 'bibtex_files/WRR/savedrecs-2.bib',
 'bibtex_files/WRR/savedrecs-21.bib',
 'bibtex_files/WRR/savedrecs-35.bib',
 'bibtex_files/WRR/savedrecs-23.bib',
 'bibtex_files/WRR/savedrecs-37.bib',
 'bibtex_files/WRR/savedrecs-36.bib',
 'bibtex_files/WRR/savedrecs-22.bib',
 'bibtex_files/WRR/savedrecs-26.bib',
 'bibtex_files/WRR/

In [32]:
# init storage
columns = ['DOI', 'Year', 'Journal', 'Title', 'Abstract', 'Affiliation']
corpus_df = pd.DataFrame(columns = columns)

In [33]:
# extract information from all bibtex entries -- store in dataframe

# loop through bib files
paper_count = -1
for working_file in tqdm(file_list):
    
    # read bibtex file by line
    with open(working_file) as fp:
        
        # read first line
        line = fp.readline()
        
        # loop until end of file
        while line:
            
            # read active line
            line = fp.readline()
            
            # find start of a new paper
            if line[:8] == '@article':
                paper_count = paper_count + 1
            
            # pull components
            if line[:5] == 'Title':
                title = line.split('{{')[1].split('}}')[0]
                corpus_df.loc[paper_count, 'Title'] = title
            if line[:7] == 'Journal':
                journal = line.split('{{')[1].split('}}')[0]
                corpus_df.loc[paper_count, 'Journal'] = journal
            if line[:8] == 'Abstract':
                text = line.split('{{')[1].split('\n')[0]
            
                # read until end of abstract (can be multi-line)
                end_of_abstract = False
                while not end_of_abstract:
                    line = fp.readline()
                    if len(line.split('}}')) == 1:
                        text = text + line.split('}}')[0].split('\n')[0]
                    else:
                        text = text + line.split('}}')[0].split('\n')[0]
                        end_of_abstract = True
                corpus_df.loc[paper_count, 'Abstract'] = text

            if line[:3] == 'DOI':
                doi = line.split('{{')[1].split('}}')[0]
                corpus_df.loc[paper_count, 'DOI'] = doi
            if line[:4] == 'Year':
                year = line.split('{{')[1].split('}}')[0]
                corpus_df.loc[paper_count, 'Year'] = year
            if line[:17] == 'Early Access Date':
                year = line.split('{{')[1].split('}}')[0][-4:]
                corpus_df.loc[paper_count, 'Year'] = year
            if line[:11] == 'Affiliation':
                affiliation = line.split('{{')[1].split('}}')[0]
                affiliation1 = affiliation.split('.')[0]
                affiliation2 = affiliation1.split(',')[-1]
                affiliation3 = affiliation2.split(' ')[-1]
                test_list = ['1','2','3','4','5','6','7','8','9','0','ny','d','wa','j','nm','tn','pa','dc','c','m','h','f','st','s','sc','r','e','b']
  
                #Setting all the zipcodes to US of A!            
                if [ele for ele in test_list if(ele in affiliation3)]:
                    affiliation3 = 'USA'
                corpus_df.loc[paper_count, 'Affiliation'] = affiliation3
            
    # close file
    fp.close()



  0%|          | 0/128 [00:00<?, ?it/s]

  1%|          | 1/128 [00:00<01:24,  1.50it/s]

  2%|▏         | 2/128 [00:00<01:07,  1.86it/s]

  2%|▏         | 3/128 [00:01<01:22,  1.51it/s]

  3%|▎         | 4/128 [00:03<01:47,  1.16it/s]

  4%|▍         | 5/128 [00:05<02:38,  1.29s/it]

  5%|▍         | 6/128 [00:07<02:54,  1.43s/it]

  5%|▌         | 7/128 [00:09<03:17,  1.64s/it]

  6%|▋         | 8/128 [00:11<03:27,  1.73s/it]

  7%|▋         | 9/128 [00:14<04:11,  2.11s/it]

  8%|▊         | 10/128 [00:17<04:32,  2.31s/it]

  9%|▊         | 11/128 [00:19<04:48,  2.46s/it]

  9%|▉         | 12/128 [00:23<05:26,  2.81s/it]

 10%|█         | 13/128 [00:29<06:57,  3.63s/it]

 11%|█         | 14/128 [00:30<05:28,  2.88s/it]

 12%|█▏        | 15/128 [00:34<06:02,  3.21s/it]

 12%|█▎        | 16/128 [00:40<07:45,  4.16s/it]

 13%|█▎        | 17/128 [00:40<05:35,  3.02s/it]

 14%|█▍        | 18/128 [00:46<06:49,  3.73s/it]

 15%|█▍        | 19/128 [00:51<07:21,  4.05s/it]

 16%|█▌        |

In [39]:
corpus_df.Affiliation.loc[0]

'CANADA'

In [40]:
# pickle the output
corpus_df.to_pickle("data/raw_corpus.pkl")

PermissionError: [Errno 13] Permission denied: 'data/raw_corpus.pkl'

In [ ]:
# convert abstracts to strings
# required to use `re` commands
data = corpus_df['Abstract'].apply(str)

In [ ]:
# manually clean the abstracts

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', text) for text in data]

# Remove new line characters
data = [re.sub('\s+', ' ', text) for text in data]

# Remove distracting single quotes
data = [re.sub("\'", "", text) for text in data]

# Hand clean everything we notice
data = [re.sub('ﬂ', 'fl', text) for text in data]
data = [re.sub('[ü←öä∀↓玉成‘’“”…©@-_/≤≥−–∂◦‡×γ∞∼±∞→⇔¨þ´≡íáˇý°φτττ¼.•˜ϕˆ˜°Á¯πłłβκßθıﬁ‰:;ψδ∈ρÞωÀηµεψ∝ðξα²®≈λσ¢·∗¤√]', '', text) for text in data]
data = [re.sub('fig', '', text) for text in data]
data = [re.sub('use', '', text) for text in data]
data = [re.sub('also', '', text) for text in data]
data = [re.sub('show', '', text) for text in data]
data = [re.sub('even', '', text) for text in data]
data = [re.sub('give', '', text) for text in data]
data = [re.sub('versant', '', text) for text in data]
data = [re.sub('statistique', '', text) for text in data]
data = [re.sub('des_crue', '', text) for text in data]
data = [re.sub('pour', '', text) for text in data]
data = [re.sub('alor', '', text) for text in data]
data = [re.sub('measurement', '', text) for text in data]
data = [re.sub('result', '', text) for text in data]
data = [re.sub('method', '', text) for text in data]
data = [re.sub('study', '', text) for text in data]
data = [re.sub('estimate', '', text) for text in data]
data = [re.sub('set', '', text) for text in data]
data = [re.sub('function', '', text) for text in data]
data = [re.sub('test', '', text) for text in data]
data = [re.sub('result', '', text) for text in data]
data = [re.sub('equation', '', text) for text in data]
data = [re.sub('mean', '', text) for text in data]
data = [re.sub('total', '', text) for text in data]
data = [re.sub('des_crue', '', text) for text in data]
data = [re.sub('mazone_obido', '', text) for text in data]
data = [re.sub('resultat', '', text) for text in data]
data = [re.sub('bido', '', text) for text in data]
data = [re.sub('debit', '', text) for text in data]
data = [re.sub('crue', '', text) for text in data]
data = [re.sub('include', '', text) for text in data]
data = [re.sub('faible', '', text) for text in data]
data = [re.sub('toujour', '', text) for text in data]
data = [re.sub('saturee', '', text) for text in data]
data = [re.sub('donnee', '', text) for text in data]
data = [re.sub('etude', '', text) for text in data]
data = [re.sub('rance', '', text) for text in data]
data = [re.sub('time', '', text) for text in data]
data = [re.sub('consist', '', text) for text in data]
data = [re.sub('update', '', text) for text in data]
data = [re.sub('opyright_ohn', '', text) for text in data]
data = [re.sub('copyright', '', text) for text in data]

In [ ]:
# change each document to a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Must run this command separately, but only once: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized[:2]

In [ ]:
# Save the lemmatized data
with open("data/data_lemmatized.pkl", 'wb') as f:
     pickle.dump(data_lemmatized, f)

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

In [ ]:
# Save the corpus and the id2word map
with open("data/cleaned_corpus.pkl", 'wb') as f:
     pickle.dump(corpus, f)
with open("data/id2word.pkl", 'wb') as f:
     pickle.dump(id2word, f)

In [ ]:
# Human readable format of document-term-frequency matrix
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

In [ ]:
# Create corpus figure

# unique journals
journals = np.unique(corpus_df['Journal'])
years = np.unique(corpus_df['Year'])

# init storage
corpus_distribution = np.full([nJournals,nYears], np.nan)

# count per journal per year
for journal in journals:
    for year in years:
        print(np.where( corpus_df['Year'] == year and corpus_df['Journal'] == journal))